In [32]:
# Load dependencies
from pymongo import MongoClient
import pandas as pd
import numpy as np
import plotly.express as px
import os

In [ ]:
# Step 1: Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["music_analysis"]
collection = db["songs"]

In [ ]:
# Loading the CSV file 
file_path = '~/Desktop/project-3/Dataset/songs_normalize.csv'

# Load the dataset
songs_df = pd.read_csv(file_path)

In [ ]:
# Cleaning up the Data - checking for missing values 
missing_values = songs_df.isnull().sum()
print("Missing Values:\n", missing_values)

# Drop rows with missing values 
songs_df_cleaned = songs_df.dropna()

# Verfiy if missing values have been dropped 
print("Missing Values After Cleanup:\n", songs_df_cleaned.isnull().sum())

In [ ]:
# Standardizing Genere Names - Splitting multi-genre 
songs_df_cleaned['genre'] = songs_df_cleaned['genre'].apply(lambda x: x.split(','[0].strip()))
print(songs_df_cleaned['genre'].value_counts())

In [ ]:
# Standardize text in artist, song, and genre columns
songs_df_cleaned['artist'] = songs_df_cleaned['artist'].str.lower().str.strip()
songs_df_cleaned['song'] = songs_df_cleaned['song'].str.lower().str.strip()

In [ ]:
# Normalize genre names by keeping only the first genre (if multiple genres are listed)
# This handles both cases: if the genre is a list or a string
songs_df_cleaned['genre'] = songs_df_cleaned['genre'].apply(lambda x: x[0] if isinstance(x, list) else x.split(',')[0].strip())

# Check the cleaned genre column
print(songs_df_cleaned['genre'].head())

In [ ]:
# Remove duplicate rows
songs_df_cleaned = songs_df_cleaned.drop_duplicates()

In [ ]:
# Normalize text by converting artist and song names to lowercase and stripping extra spaces
songs_df_cleaned['artist'] = songs_df_cleaned['artist'].str.lower().str.strip()
songs_df_cleaned['song'] = songs_df_cleaned['song'].str.lower().str.strip()

In [ ]:
# Convert duration from milliseconds to minutes
songs_df_cleaned['duration_minutes'] = songs_df_cleaned['duration_ms'] / 60000

In [ ]:
# Check for any invalid years (e.g., negative values or far future years)
songs_df_cleaned = songs_df_cleaned[songs_df_cleaned['year'].between(1900, 2024)]

In [ ]:
# Remove outliers beyond a certain threshold (for example, using 1.5 * IQR rule)
Q1 = songs_df_cleaned[['tempo', 'energy', 'loudness']].quantile(0.25)
Q3 = songs_df_cleaned[['tempo', 'energy', 'loudness']].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds for acceptable values
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove rows that contain outliers
songs_df_cleaned = songs_df_cleaned[~((songs_df_cleaned[['tempo', 'energy', 'loudness']] < lower_bound)|
                                      (songs_df_cleaned[['tempo', 'energy', 'loudness']] > upper_bound)).any(axis=1)]

In [ ]:
# Display the cleaned data (first 5 rows and summary)
print(songs_df_cleaned.head())
print("\nSummary of the cleaned data:\n", songs_df_cleaned.describe())
print("\nRemaining missing values (should be 0 for all columns):\n", songs_df_cleaned.isnull().sum())
print(f"\nNumber of duplicates after cleaning: {songs_df_cleaned.duplicated().sum()}")

In [ ]:
# Save the cleaned dataset to a new CSV file
songs_df_cleaned.to_csv('cleaned_songs_data.csv', index=False)

Additional cleaning

In [34]:
df = pd.read_csv('Downloads/cleaned_songs_data.csv', delimiter = ',', quoting = 0)

In [36]:
print(df['artist'].head())

0    britney spears
1         blink-182
2        faith hill
3          bon jovi
4            *nsync
Name: artist, dtype: object


In [38]:
max_danceability = df['danceability'].max()
df['danceability'] = (df['danceability'] / max_danceability) * 100

In [40]:
max_energy = df['energy'].max()
df['energy'] = (df['energy'] / max_energy) * 100

In [42]:
max_speechiness = df['speechiness'].max()
df['speechiness'] = (df['speechiness'] / max_speechiness) * 100

In [44]:
max_acousticness = df['acousticness'].max()
df['acousticness'] = (df['acousticness'] / max_acousticness) * 100

In [46]:
max_liveness = df['liveness'].max()
df['liveness'] = (df['liveness'] / max_liveness) * 100

In [48]:
max_valence = df['valence'].max()
df['valence'] = (df['valence'] / max_valence) * 100

In [50]:
null_counts = df.isnull().sum()
print("Null value counts:")
print(null_counts)

# Check for duplicates
duplicates = df[df.duplicated()]

if not duplicates.empty:
    print("Duplicate rows found:")
    print(duplicates)

Null value counts:
artist              0
song                0
duration_ms         0
explicit            0
year                0
popularity          0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
genre               0
duration_minutes    0
dtype: int64


In [52]:
print(df['genre'].unique())

['pop' 'rock' 'hip hop' 'Dance/Electronic' 'metal' 'R&B' 'Folk/Acoustic'
 'set()' 'World/Traditional' 'latin' 'country' 'easy listening']


In [54]:
df = df[df['genre'] != 'set()']

In [56]:
df.reset_index(drop=True, inplace=True)

In [62]:
df.to_csv('project_3_clean.csv', index=False)